In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

from IPython.display import clear_output
from datetime import datetime

In [2]:
param_date = '20240726'

In [3]:
import os

list_file = sorted(list(os.walk(f'./{param_date}/6.part_number_2'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['group_1.Apparel & Gifts.xlsx',
 'group_10.Electrical-Switch & Relay-serial_1.xlsx',
 'group_10.Electrical-Switch & Relay-serial_2.xlsx',
 'group_10.Electrical-Switch & Relay-serial_3.xlsx',
 'group_11.Engine-serial_1.xlsx',
 'group_11.Engine-serial_10.xlsx',
 'group_11.Engine-serial_2.xlsx',
 'group_11.Engine-serial_3.xlsx',
 'group_11.Engine-serial_4.xlsx',
 'group_11.Engine-serial_5.xlsx',
 'group_11.Engine-serial_6.xlsx',
 'group_11.Engine-serial_7.xlsx',
 'group_11.Engine-serial_8.xlsx',
 'group_11.Engine-serial_9.xlsx',
 'group_12.Exhaust & Emission-serial_1.xlsx',
 'group_12.Exhaust & Emission-serial_2.xlsx',
 'group_12.Exhaust & Emission-serial_3.xlsx',
 'group_12.Exhaust & Emission-serial_4.xlsx',
 'group_12.Exhaust & Emission-serial_5.xlsx',
 'group_12.Exhaust & Emission-serial_6.xlsx',
 'group_13.Fuel & Air-serial_1.xlsx',
 'group_13.Fuel & Air-serial_2.xlsx',
 'group_13.Fuel & Air-serial_3.xlsx',
 'group_13.Fuel & Air-serial_4.xlsx',
 'group_13.Fuel & Air-serial_5.xlsx',
 

In [ ]:
for file in list_file:

    # = = = = = = = = = = = = = = =

    import pandas as pd

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./{param_date}/6.part_number_2/{file}',
                           header=0,
                           dtype=str).fillna('')

    input_['Group No'] = input_['Group No'].astype(int)
    input_['Type No'] = input_['Type No'].astype(int)
    input_['Manufacturer No'] = input_['Manufacturer No'].astype(int)
    input_['Part No'] = input_['Part No'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =

    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            import requests
            import json

            import sys
            sys.path.append('../../00.Tools')
            from crawler_configuration import get_header, get_proxy

            data = {'dopartsearch': '1',
                    'partsearch[partnum][partsearch_007]': input_.loc[a, 'Part Number'],
                    'partsearch[manufacturer][partsearch_007]': input_.loc[a, 'Manufacturer'],
                    'partsearch[partgroup][partsearch_007]': input_.loc[a, 'Group'],
                    'partsearch[parttype][partsearch_007]': input_.loc[a, 'Type Code'],
                    'partsearch[do][partsearch_007]': '1',
                    'func': 'sendparttabsearch',
                    'giveback': json.dumps({}),
                    'api_json_request': '1'}

            try:
                b = 0
                while True:

                    while True:
                        b += 1
                        try:
                            resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                                 data=data,
                                                 headers=get_header(),
                                                 proxies=get_proxy(),
                                                 timeout=(10, 10))
                            break
                        except:
                            continue

                    if resp.status_code == 200:
                        json_ = resp.json()
                        if 'redirect_to_url' in json_:
                            continue
                        elif not resp.json()['giveback'] and 'collected_javascript' in json_:
                            break

                # = = = = = = = = = = = = = = =

                from bs4 import BeautifulSoup
                from lxml import etree

                soup = BeautifulSoup(json_['searchnoderesults'], 'lxml')
                html = etree.HTML(str(soup))

                # = = = = = = = = = = = = = = =

                list_tbody = html.xpath('//tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-a-1") or contains(@class, "altrow-a-0"))]')

                # = = = = = = = = = = = = = = =

                list_json = [json.loads(tbody.xpath('./tr[1]/td[contains(@class, "listing-inner-content")]/input[contains(@id, "listing_data_essential[__GIP__")]/@value')[0]) for tbody in list_tbody]

                # = = = = = = = = = = = = = = =

                import numpy as np

                array_index_type_code = np.where(np.array([json_['parttype'] for json_ in list_json]) == input_.loc[a, 'Type Code'])

                # = = = = = = = = = = = = = = =

                list_key = [json_['partkey'] for json_ in list_json]

                # = = = = = = = = = = = = = = =

                list_json = [json.loads(tbody.xpath('./tr[1]/td/input[contains(@id, "listing_data_supplemental[")]/@value')[0]) for tbody in list_tbody]

                # = = = = = = = = = = = = = = =

                array_index_part_number = np.where(np.array([json_['partnumber'] for json_ in list_json]) == input_.loc[a, 'Part Number'])

                # = = = = = = = = = = = = = = =

                array_index_manufacturer = np.where(np.array([json_['catalogname'] for json_ in list_json]) == input_.loc[a, 'Manufacturer'])

                # = = = = = = = = = = = = = = =

                list_parameter = [json_['paramdesc'].strip() for json_ in list_json]

                # = = = = = = = = = = = = = = =

                array_index_part = np.intersect1d(np.intersect1d(array_index_type_code, array_index_part_number), array_index_manufacturer)

                # = = = = = = = = = = = = = = =

                if len(array_index_part) != 1:
                    raise

                # = = = = = = = = = = = = = = =

                part_index = array_index_part[0]

                # = = = = = = = = = = = = = = =

                list_note = list_tbody[part_index].xpath('./tr[1]/td/div[@class="listing-text-row-moreinfo-truck"]/span[@class="span-link-underline-remover"]/text()')
                note = '\n'.join([note.strip() for note in list_note])

                # = = = = = = = = = = = = = = =

                list_info = list_tbody[part_index].xpath('./tr[1]/td/div[@class="listing-text-row-moreinfo-truck"]/a[@class="ra-btn ra-btn-moreinfo"]/@href')
                info = list_info[0].strip() if list_info and list_info[0].startswith('https://www.rockauto.com/en/moreinfo.php?') else ''

                # = = = = = = = = = = = = = = =

                import re

                note_2 = etree.tostring(list_tbody[part_index].xpath('./tr[1]/td/div[@class="listing-text-row"]')[0]).decode('utf-8')
                note_2 = re.sub(r'<.*?>', '', note_2.strip())
                note_2 = re.sub(r' +', ' ', note_2.strip())

                # = = = = = = = = = = = = = = =

                list_oem = list_tbody[part_index].xpath('./tr[1]/td/span[contains(@title, "Replaces these Alternate")]/text()')
                oem = ','.join(list_oem)
                oem = ';'.join([oem_split.strip() for oem_split in oem.split(',')])

                # = = = = = = = = = = = = = = =

                list_src = json.loads(list_tbody[part_index].xpath('./tr[1]/td/div/input[contains(@id, "jsninlineimg[")]/@value')[0])['Slots'] if list_tbody[part_index].xpath('./tr[1]/td/div/input[contains(@id, "jsninlineimg[")]/@value') else []

                # = = = = = = = = = = = = = = =

                dict_src = {}
                for i, src in zip(range(len(list_src)), list_src):
                    dict_src[str(i)] = 'https://www.rockauto.com' + src['ImageData']['Full'].strip()

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame([{'Group No': input_.loc[a, 'Group No'],
                                         'Group': input_.loc[a, 'Group'],
                                         'Type No': input_.loc[a, 'Type No'],
                                         'Type': input_.loc[a, 'Type'],
                                         'Type Code': input_.loc[a, 'Type Code'],
                                         'Manufacturer No': input_.loc[a, 'Manufacturer No'],
                                         'Manufacturer': input_.loc[a, 'Manufacturer'],
                                         'Tab 2 Url': input_.loc[a, 'Tab 2 Url'],
                                         'Part No': input_.loc[a, 'Part No'],
                                         'Part Number': input_.loc[a, 'Part Number'],
                                         'Url': input_.loc[a, 'Url'],
                                         'Key': list_key[part_index],
                                         'JOIN_MPNTCK': f'''{input_.loc[a, 'Manufacturer']};{input_.loc[a, 'Part Number']};{input_.loc[a, 'Type Code']};{list_key[part_index]}''',
                                         'Tab 2 Note 1': note,
                                         'Tab 2 Note 2': note_2,
                                         'Parameter': list_parameter[part_index],
                                         'OEM': oem,
                                         'Info': info,
                                         'Json_Src': json.dumps(dict_src)}])

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                if crawler_status == 'error':
                    df_temp = pd.DataFrame([{'Group No': input_.loc[a, 'Group No'],
                                             'Group': input_.loc[a, 'Group'],
                                             'Type No': input_.loc[a, 'Type No'],
                                             'Type': input_.loc[a, 'Type'],
                                             'Type Code': input_.loc[a, 'Type Code'],
                                             'Manufacturer No': input_.loc[a, 'Manufacturer No'],
                                             'Manufacturer': input_.loc[a, 'Manufacturer'],
                                             'Tab 2 Url': input_.loc[a, 'Tab 2 Url'],
                                             'Part No': input_.loc[a, 'Part No'],
                                             'Part Number': input_.loc[a, 'Part Number'],
                                             'Url': input_.loc[a, 'Url']}])

                    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()

            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'Type No']}.{input_.loc[a, 'Type']} > {input_.loc[a, 'Manufacturer No']}.{input_.loc[a, 'Manufacturer']} > {input_.loc[a, 'Part No']}.{input_.loc[a, 'Part Number']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(7):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['Group No', 'Type No', 'Manufacturer No', 'Part No'],
                                                    ascending=[True, True, True, True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./{param_date}/7.part_1/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['Group No', 'Type No', 'Manufacturer No', 'Part No'],
                                                ascending=[True, True, True, True],
                                                ignore_index=True)
        output_error.to_excel(f'''./{param_date}/7.part_1/0.error/{file.removesuffix('.xlsx')}-part_1_error.xlsx''', index=False)
        print('爬虫存在error')
        print()
    print('Done ~')

[状态：ok，尝试次数：26] - 31.07% - group_16.Heat & Air Conditioning-serial_4 > 68.A/C Refrigerant Oil > 6.MOTORCRAFT > 4.YN50
[剩余数量：13000] - [当前时间：08:30:44]

[状态：ok，尝试次数：17] - 31.07% - group_16.Heat & Air Conditioning-serial_4 > 68.A/C Refrigerant Oil > 4.GPD > 2.8011250
[剩余数量：12999] - [当前时间：08:30:45]

[状态：ok，尝试次数：31] - 31.07% - group_16.Heat & Air Conditioning-serial_4 > 68.A/C Refrigerant Oil > 7.UAC > 1.RO0001B
[剩余数量：12998] - [当前时间：08:30:45]

[状态：ok，尝试次数：15] - 31.07% - group_16.Heat & Air Conditioning-serial_4 > 68.A/C Refrigerant Oil > 7.UAC > 7.RO0902B
[剩余数量：12997] - [当前时间：08:30:45]

[状态：ok，尝试次数：2] - 31.07% - group_16.Heat & Air Conditioning-serial_4 > 69.A/C Refrigerant Temperature Sensor > 2.AUTOTECNICA > 1.GM1318664
[剩余数量：12996] - [当前时间：08:30:47]

[状态：ok，尝试次数：5] - 31.07% - group_16.Heat & Air Conditioning-serial_4 > 69.A/C Refrigerant Temperature Sensor > 4.GM GENUINE > 1.1550452
[剩余数量：12995] - [当前时间：08:30:49]

[状态：ok，尝试次数：15] - 31.07% - group_16.Heat & Air Conditioning-serial_4 > 69.A